In [1]:
import requests
import folium
from sklearn.cluster import KMeans
from IPython.display import display
import numpy as np

# Define your API key
api_key = 'AlzaSyA6OARNW6uKqx_gFXorv0kXGaGZSx7ozMR'

def get_location_from_name(place_name):
    url = f'https://maps.gomaps.pro/maps/api/place/textsearch/json?query={place_name}&key={api_key}'
    response = requests.get(url)
    data = response.json()
    
    if data['results']:
        location = data['results'][0]['geometry']['location']
        return location['lat'], location['lng']
    return None, None

def get_nearby_hotels(lat, lng, radius=2000):
    url = f'https://maps.gomaps.pro/maps/api/place/nearbysearch/json?location={lat},{lng}&radius={radius}&type=lodging&key={api_key}'
    response = requests.get(url)
    
    if response.status_code != 200:
        print("Error: Unable to fetch data from API.")
        return []

    data = response.json()
    hotels = []
    for result in data.get('results', []):
        if result.get('business_status') == 'OPERATIONAL' and 'lodging' in result.get('types', []):
            hotels.append({
                'name': result['name'],
                'address': result.get('vicinity', 'Address not available'),
                'price_level': result.get('price_level', 'Unknown'),
                'rating': result.get('rating', 'N/A'),
                'lat': result['geometry']['location']['lat'],  # Save latitude
                'lng': result['geometry']['location']['lng']   # Save longitude
            })
    return hotels

def get_hotels_near_closest_tourist_spot(tourist_places):
    for lat, lng in tourist_places:
        hotels = get_nearby_hotels(lat, lng)
        if hotels:
            return hotels
    return []

def get_hotels_for_all_clusters(centroids):
    for lat, lng in centroids:
        hotels = get_nearby_hotels(lat, lng)
        if hotels:
            return hotels
    return []



def calculate_kmeans_clusters(tourist_places, n_clusters=1):
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=0).fit(tourist_places)
    return kmeans.cluster_centers_

def find_best_hotels(tourist_places, num_clusters=1, initial_radius=5000):
    # Step 1: Calculate centroids using K-means
    centroids = calculate_kmeans_clusters(tourist_places, n_clusters=num_clusters)

    # Step 2: Search for hotels near each centroid
    all_hotels = []
    for lat, lng in centroids:
        hotels = get_nearby_hotels(lat, lng, initial_radius)
        if hotels:
            all_hotels.extend(hotels)  # Collect all hotels

    # Step 3: Search for hotels near the closest tourist spot
    if not all_hotels:
        all_hotels = get_hotels_near_closest_tourist_spot(tourist_places)

    if not all_hotels:
        return suggest_backup_hotels(), centroids

    return all_hotels, centroids

def plot_hotels_on_map(hotels, centroids):
    
    # Create map centered at the first centroid
    m = folium.Map(location=centroids[0], zoom_start=12)
    
    # Sort hotels by rating (highest first)
    sorted_hotels = sorted(hotels, key=lambda x: x.get('rating', 0), reverse=True)
    
    # Select top 10 hotels
    top_10_hotels = sorted_hotels[:10]
    
    # Add markers for the top 10 hotels
    for hotel in top_10_hotels:
        folium.Marker(
            location=[hotel['lat'], hotel['lng']],  # Use lat/lng from hotel data
            popup=f"Name: {hotel['name']}\nAddress: {hotel['address']}\nRating: {hotel['rating']}",
            icon=folium.Icon(color='green')
        ).add_to(m)
    
    # Calculate the area with the most hotels (highest hotel density)
    hotel_lats = [hotel['lat'] for hotel in hotels]
    hotel_lngs = [hotel['lng'] for hotel in hotels]
    
    # Find the center of the highest density of hotels
    avg_lat = np.mean(hotel_lats)
    avg_lng = np.mean(hotel_lngs)
    
    # Highlight the area around the most concentrated hotels with a circle
    folium.Circle(
        location=[avg_lat, avg_lng],
        radius=2000,  # Adjust the radius to your preference
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.3
    ).add_to(m)

    # Add a marker for the centroid (Best Area to Stay)
    for lat, lng in centroids:
        folium.Marker(
            location=[lat, lng],
            popup="Best Area to Stay",
            icon=folium.Icon(color='red')
        ).add_to(m)

    return m

# Example usage with location names instead of lat/long
tourist_place_names = ["khandagiri, bhubneswar, india", "master canteen, bhubneswar, india","utkal university,bhubneswar,india"]  # Example places

# Retrieve lat/lng for each tourist spot
tourist_places = []
for place_name in tourist_place_names:
    lat, lng = get_location_from_name(place_name)
    if lat is not None and lng is not None:
        tourist_places.append((lat, lng))

# Check if any valid tourist places were found
if tourist_places:
    hotels, centroids = find_best_hotels(tourist_places, num_clusters=1)  # Get hotels and centroids
    
    # Display the details of the hotels
    print("Suggested Hotels:")
    for hotel in hotels:
        print(f"Name: {hotel['name']}")
        print(f"Address: {hotel['address']}")
        print(f"Price Level: {hotel['price_level']}")
        print(f"Rating: {hotel['rating']}")
        print(f"Latitude: {hotel['lat']}, Longitude: {hotel['lng']}")
        print("-" * 40)
    
    # Plotting the hotels on the map with the best area highlighted
    m = plot_hotels_on_map(hotels, centroids)
    display(m)
else:
    print("No valid tourist places found.")


C:\Users\USER\anaconda3\an\Lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\USER\anaconda3\an\Lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(


Suggested Hotels:
Name: Trident Hotel Bhubaneswar
Address: CB-1, Nayapalli, Bhubaneswar
Price Level: Unknown
Rating: 4.5
Latitude: 20.3023545, Longitude: 85.8212676
----------------------------------------
Name: Hotel Pushpak
Address: Plot No 68, Kalpana Square, Puri - Cuttack Road, Buddha Nagar, Bhubaneswar
Price Level: Unknown
Rating: 4.3
Latitude: 20.2589902, Longitude: 85.84148669999999
----------------------------------------
Name: Hotel Pal Heights
Address: J-7, Jayadev Vihar-Ekamra Kanana Road, Jayadev Vihar, Bhubaneswar
Price Level: Unknown
Rating: 4.1
Latitude: 20.2985896, Longitude: 85.8232381
----------------------------------------
Name: Radhika Resort Bhubaneswar
Address: A17, Bhouma Nagar, Bhubaneswar
Price Level: Unknown
Rating: 4
Latitude: 20.2766895, Longitude: 85.8327561
----------------------------------------
Name: Hotel Sandy's Tower
Address: Plot No. P-1 & P-1/A, Jaydev Vihar, Bhubaneswar
Price Level: Unknown
Rating: 4.3
Latitude: 20.3078733, Longitude: 85.8216076